In [4]:
import pandas as pd
import numpy as np
import glob

In [430]:
file=glob.glob('../train/*.csv')
file

['../train/train_三瓦窑.csv',
 '../train/train_龙泉驿区政府.csv',
 '../train/train_金泉两河.csv',
 '../train/train_灵岩寺.csv',
 '../train/train_君平街.csv',
 '../train/train_十里店.csv',
 '../train/train_沙河铺.csv',
 '../train/train_大石西路.csv']

In [81]:
LEVEL_mapping = {
    '六级': 6,
    '五级': 5,      
    '四级': 4,
    '三级': 3,
    '二级': 2,
    '一级': 1}

datas=list()
for i in file:
    data=pd.read_csv(i)
    data=data.drop(['UPDATE_TIME_','MN_','SOURCE_','CATEGORY_','COLOR_','STATE_'],axis=1)
    data['LEVEL_']=data['LEVEL_'].map(LEVEL_mapping)
    #data=pd.get_dummies(data)
    data=data.set_index('DATA_TIME_')
    datas.append(data)

In [683]:
#生成日期
date=list()
for i in range(12):
    month=[31,28,31,30,24,30,31,31,30,31,30,24]
    for j in range(1,month[i]+1):
        for k in range(24):
            a='2017/{0}/{1} {2:02d}:00:00'.format(i+1,j,k)
            date.append(a)

In [684]:
date=pd.DataFrame(date)
#ab=data.set_index('DATA_TIME_')
date=date.set_index(0)
X=date.join(pd.concat([datas[0],datas[2],datas[3],datas[4],datas[5],datas[6],datas[7]],axis=1,sort=True))

In [223]:
add=list()
addday=list()
for i in range(8424):
    add.append(i%24+1)
    addday.append(int(i%168/(24)))
X['hour']=add
X['day']=addday
X = X.reset_index(drop=True)

Y=X['AQI_']
X.columns=range(X.shape[1])
Y.columns=range(Y.shape[1])

In [224]:
def delcolumns(X):
    indx=list()
    for i in range(X.shape[1]):
        a=X.iloc[:,i].isnull().sum()
        if a/X.shape[0]>0.5:
            indx.append(i)
    X=X.drop(X.columns[indx],axis=1)
    return X
X=delcolumns(X)

In [185]:
def hanNaN(X):
    for feature_num,feature in enumerate(X.columns):
        feature_index=X[X[feature].isnull()].index
        for i in range(len(feature_index)):
            if feature_index[i]==0:
                j=feature_index[i]+1
                while j in feature_index:
                    j=j+1
                X.iloc[feature_index[i],feature_num]=X.iloc[j,feature_num]
            else:
                j=feature_index[i]

                while j in feature_index:
                    j=j-1
                    if j==0:
                        break
                X.iloc[feature_index[i],feature_num]=X.iloc[j,feature_num]
    return X
XX=hanNaN(X)
X=Y
Y=hanNaN(X)

In [186]:
XX=pd.get_dummies(XX)

In [227]:
XX.shape

(8424, 153)

In [228]:
Y.shape

(8424, 7)

In [213]:
X_train=XX.iloc[:130*24,:]
y_train=Y.iloc[24*7:137*24,:]
X_test=XX.iloc[130*24:137*24,:]
y_test=Y.iloc[137*24:144*24,:]

In [194]:
from sklearn.utils import shuffle
from sklearn.model_selection import ParameterGrid
from sklearn import datasets, ensemble

In [214]:
X_train, y_train = shuffle(X_train, y_train)

In [215]:
params = {'learning_rate': 0.01,
          'loss': 'ls'}
parameters={'max_depth':range(1,10),'n_estimators':[10,30,50,70,100,150,200],'min_samples_split':range(2,10)}
reg = ensemble.GradientBoostingRegressor(**params)

In [216]:
best_score=float('inf')
for g in ParameterGrid(parameters):
    reg.set_params(**g)
    reg.fit(X_train,y_train[0])
    y_pred=reg.predict(X_test)
    # save if best
    score = np.sqrt(((y_pred - y_test[0]) ** 2).mean())
    if score < best_score:
        best_score = score
        best_grid = g

print("score: %0.5f" % best_score)
print("Grid:", best_grid)

score: 25.53470
Grid: {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 200}


In [449]:
X_train_=XX.iloc[:137*24,:]
y_train_=Y.iloc[24*7:144*24,:]
X_test_=XX.iloc[137*24:144*24,:]

X_train_, y_train_ = shuffle(X_train_, y_train_)

In [450]:
params_best = {'max_depth':5,
               'min_samples_split':5,
               'n_estimators':200,
               'learning_rate':0.01,
               'loss':'ls'}
y_pred1=list()
for i in range(7):
    reg = ensemble.GradientBoostingRegressor(**params_best)
    reg.fit(X_train_, y_train_[i])
    y_pred1.append(reg.predict(X_test_))
    

In [451]:
X_train_2=XX.iloc[144*24:(-7*24),:]
y_train_2=Y.iloc[151*24:,:]
X_test_2 =XX.iloc[-7*24:,:]

X_train_2, y_train_2 = shuffle(X_train_2, y_train_2)

In [452]:
params_best = {'max_depth':5,
               'min_samples_split':5,
               'n_estimators':200,
               'learning_rate':0.01,
               'loss':'ls'}
y_pred2=list()
for i in range(7):
    reg = ensemble.GradientBoostingRegressor(**params_best)
    reg.fit(X_train_2, y_train_2[i])
    y_pred2.append(reg.predict(X_test_2))

In [260]:
datas[1].loc['2017/10/30 16:00:00']

C_SO2_1H                  5
I_SO2_1H                  2
C_NO2_1H                 16
I_NO2_1H                  8
C_PM10_1H                12
C_PM10_24H_MV           NaN
I_PM10_24H_MV            12
C_CO1_1H               0.75
I_CO1_1H                  8
C_O3_1H                  21
I_O3_1H                   7
C_O3_8H_MV              NaN
I_O3_8H_MV              NaN
C_PM25_1H                 8
C_PM25_24H_MV           NaN
I_PM25_24H_MV            12
AQI_                     12
PRIMARY_POLLUTANTS_       -
LEVEL_                    1
Name: 2017/10/30 16:00:00, dtype: object

In [355]:
dateLong=date.loc['2017/10/30 14:00:00':]
XLong=dateLong.join(pd.concat(datas,axis=1,sort=True))

In [356]:
add=list()
addday=list()
for i in range(XLong.shape[0]):
    add.append(i%24+15)
    addday.append(int(i%168/(24)))
XLong['hour']=add
XLong['day']=addday
XLong = XLong.reset_index(drop=True)

YLong=XLong['AQI_']
XLong.columns=range(XLong.shape[1])
YLong.columns=range(YLong.shape[1])

In [358]:
XLong=delcolumns(XLong)
XL=hanNaN(XLong)
YL=hanNaN(YLong)
XL=pd.get_dummies(XL)

In [453]:
X_train_3=XL.iloc[:(-7*24),:]
y_train_3=YL.iloc[7*24:,1]
X_test_3 =XL.iloc[-7*24:,:]

X_train_3, y_train_3 = shuffle(X_train_3, y_train_3)

In [454]:
params_best = {'max_depth':5,
               'min_samples_split':5,
               'n_estimators':200,
               'learning_rate':0.01,
               'loss':'ls'}
reg = ensemble.GradientBoostingRegressor(**params_best)
reg.fit(X_train_3, y_train_3)
y_pred3=reg.predict(X_test_3)

In [391]:
14*24

336

In [664]:
submit=pd.read_csv('../to_predict/to_predict.csv')

In [665]:
submit.loc[submit['MN_']==51010901001,:]['DAY_HOUR'].drop_duplicates()

0       2017/5/25 00:xx
1       2017/5/25 01:xx
2       2017/5/25 02:xx
3       2017/5/25 03:xx
4       2017/5/25 04:xx
             ...       
344    2017/12/31 19:xx
345    2017/12/31 20:xx
347    2017/12/31 21:xx
348    2017/12/31 22:xx
349    2017/12/31 23:xx
Name: DAY_HOUR, Length: 336, dtype: object

In [666]:
submit['MN_'].drop_duplicates()

0       51010901001
351     51010801001
705     51010501001
1053    51010501002
1407    51010401001
1760    51010101001
2111    51010601002
2461    51011201004
Name: MN_, dtype: int64

In [667]:
MN=list()
for i in file:
    data=pd.read_csv(i)
    MN.append(data['MN_'][0])
    print(i.split('/')[2]+': ',data['MN_'][0])

train_三瓦窑.csv:  51010901001
train_龙泉驿区政府.csv:  51011201004
train_金泉两河.csv:  51010601002
train_灵岩寺.csv:  51010101001
train_君平街.csv:  51010501001
train_十里店.csv:  51010801001
train_沙河铺.csv:  51010401001
train_大石西路.csv:  51010501002


In [668]:
MN

[51010901001,
 51011201004,
 51010601002,
 51010101001,
 51010501001,
 51010801001,
 51010401001,
 51010501002]

In [460]:
pred=list()
for i in range(len(y_pred1)):
    pred.append(np.concatenate((y_pred1[i],y_pred2[i])))
pred.insert(1,y_pred3)   

In [722]:
dateNew=list()
for i in [5,12]:
    for j in range(1,8):
        for k in range(24):
            a='2017/{0}/{1} {2:02d}:00:00'.format(i,j+24,k)
            dateNew.append(a)
dateNew=pd.DataFrame(dateNew,columns=['date'])
#ab=data.set_index('DATA_TIME_')
#dateNew=dateNew.set_index(0)

In [669]:
submit.loc[submit['MN_']==51010901001]['DAY_HOUR']

0       2017/5/25 00:xx
1       2017/5/25 01:xx
2       2017/5/25 02:xx
3       2017/5/25 03:xx
4       2017/5/25 04:xx
             ...       
346    2017/12/31 20:xx
347    2017/12/31 21:xx
348    2017/12/31 22:xx
349    2017/12/31 23:xx
350    2017/12/31 23:xx
Name: DAY_HOUR, Length: 351, dtype: object

In [534]:
predList=list()
for i in submit['MN_'].drop_duplicates():
    dataIndex=submit.loc[submit['MN_']==i]['DAY_HOUR']
    dataIndex=pd.DataFrame(dataIndex).set_index('DAY_HOUR')
    for j,k in enumerate(MN):
        if k==i:
            predSub=pd.DataFrame(pred[j])
    predList.append(dataIndex.join(pd.concat([dateNew,predSub],axis=1,sort=True).set_index('date')))
    

In [605]:
su=0
for i in range(len(predList)):
    su+=predList[i].shape[0]
su

2815

In [642]:
submit=pd.concat(predList).reset_index(drop=True).reset_index().rename(columns={'index':'ID_SUBMIT',0:'AQI_'})

In [643]:
sum(submit['AQI_'].isnull())

186

In [652]:
for i in submit.loc[submit['AQI_'].isnull()].index:
    submit.iloc[i,1]=y_pred1[0].mean()

In [653]:
submit.to_csv('./submit2.csv',index=False)

In [647]:
submit.loc[submit['AQI_'].isnull()].index

Int64Index([2461, 2462, 2463, 2464, 2465, 2466, 2467, 2468, 2469, 2470,
            ...
            2637, 2638, 2639, 2640, 2641, 2642, 2643, 2644, 2645, 2646],
           dtype='int64', length=186)

In [646]:
y_pred1[0].mean()

69.1713496227487

In [651]:
submit.iloc[2647,1]

104.31459285026675

In [673]:
pd.concat([dateNew,pd.DataFrame(pred[0])],axis=1,sort=True)

,date,0
0,2017/5/25 00:xx,63.740986
1,2017/5/25 01:xx,62.363351
2,2017/5/25 02:xx,62.363351
3,2017/5/25 03:xx,60.119943
4,2017/5/25 04:xx,60.031653
...,...,...
331,2017/12/31 19:xx,105.339973
332,2017/12/31 20:xx,102.460232
333,2017/12/31 21:xx,104.652733
334,2017/12/31 22:xx,103.384086


In [689]:
dataIndex=submit.loc[submit['MN_']==51010901001]['DAY_HOUR']

In [727]:
test=list()
for i,j in enumerate(dataIndex):
    m=j.split(':')[0]+':00:00'
    test.append(m)

In [728]:
test=pd.DataFrame(test)

In [729]:
test=test.set_index(0)

In [735]:
test

""
0
2017/5/25 00:00:00
2017/5/25 01:00:00
2017/5/25 02:00:00
2017/5/25 03:00:00
2017/5/25 04:00:00
...
2017/12/31 20:00:00
2017/12/31 21:00:00
2017/12/31 22:00:00


In [738]:
test.join(pd.concat([dateNew,pd.DataFrame(pred[0])],axis=1,sort=True).set_index('date'))

,0
2017/12/25 00:00:00,130.788210
2017/12/25 01:00:00,130.788210
2017/12/25 02:00:00,101.809057
2017/12/25 03:00:00,100.791206
2017/12/25 04:00:00,113.337762
...,...
2017/5/31 19:00:00,74.712696
2017/5/31 20:00:00,60.406625
2017/5/31 21:00:00,71.406672
2017/5/31 22:00:00,90.734549


In [686]:
dataIndex

""
DAY_HOUR
2017/5/25 00:xx
2017/5/25 01:xx
2017/5/25 02:xx
2017/5/25 03:xx
2017/5/25 04:xx
...
2017/12/31 20:xx
2017/12/31 21:xx
2017/12/31 22:xx


In [739]:
test.join(pd.concat([dateNew,pd.DataFrame(pred[0])],axis=1,sort=True).set_index('date'))

,0
2017/12/25 00:00:00,130.788210
2017/12/25 01:00:00,130.788210
2017/12/25 02:00:00,101.809057
2017/12/25 03:00:00,100.791206
2017/12/25 04:00:00,113.337762
...,...
2017/5/31 19:00:00,74.712696
2017/5/31 20:00:00,60.406625
2017/5/31 21:00:00,71.406672
2017/5/31 22:00:00,90.734549


In [743]:
pd.DataFrame(range(6))

,0
0,0
1,1
2,2
3,3
4,4
5,5


In [745]:
test.join(pd.DataFrame(range(6)))

,0
2017/12/25 00:00:00,NaN
2017/12/25 01:00:00,NaN
2017/12/25 02:00:00,NaN
2017/12/25 03:00:00,NaN
2017/12/25 04:00:00,NaN
...,...
2017/5/31 19:00:00,NaN
2017/5/31 20:00:00,NaN
2017/5/31 21:00:00,NaN
2017/5/31 22:00:00,NaN
